In [1]:
import src.data as data

data.DataFetcher.main()
data.PreProcess.main(False)



Use device name: NVIDIA GeForce RTX 4090
Unpack Update Files
['/home/mengkjin/Workspace/SharedFolder/DB_updater.240408085141.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240413173912.tar']


24-04-13 17:49:28|MOD:PreProcess  |: if_train is False , Data Processing start!
24-04-13 17:49:28|MOD:PreProcess  |: 3 datas :['y', 'trade_day', 'trade_30m']


Sat Apr 13 17:49:28 2024 : All Updates Done! Cost 3.96 Secs
Sat Apr 13 17:49:28 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.49 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.38 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.96 secs
models blocks merging ...... cost 0.18 secs
y blocks process ...... cost 1.07 secs
y blocks masking ...... cost 0.06 secs
y blocks saving  ...... cost 0.12 secs
y blocks norming ...... cost 0.00 secs
Sat Apr 13 17:49:32 2024 : y finished! Cost 4.32 Seconds
Sat Apr 13 17:49:32 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.68 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.04 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.14 secs
trade_day blocks norming ...... cost 0.00 secs
Sat Apr 13 17:49:33 2024 : trade_day finished! Cost 0.92 Seconds
S

24-04-13 17:49:39|MOD:PreProcess  |: Data Processing Finished! Cost 11.16 Seconds


... cost 1.14 secs
trade_30m blocks norming ...... cost 0.00 secs
Sat Apr 13 17:49:39 2024 : trade_30m finished! Cost 5.48 Seconds


In [3]:
import socket
if socket.gethostname() == 'mengkjin-server':
    data.DataFetcher_sql.DataFetcher_sql.update_since()

Sat Apr 13 17:50:16 2024 : download since!
Connection and Factor preparation finished!
Sat Apr 13 17:50:16 2024 : sellside/haitong.hf_factors from 20240404 to 20240413, total 1 periods
Start sellside/haitong.hf_factors:20240404-20240413 
Done sellside/haitong.hf_factors:20240404-20240413 
Sat Apr 13 17:50:28 2024 : sellside/haitong.dl_factors from 20240404 to 20240413, total 1 periods
Start sellside/haitong.dl_factors:20240404-20240413 
Done sellside/haitong.dl_factors:20240404-20240413 
Sat Apr 13 17:51:59 2024 : sellside/dongfang.hfq_chars from 20240404 to 20240413, total 1 periods
Start sellside/dongfang.hfq_chars:20240404-20240413 
Done sellside/dongfang.hfq_chars:20240404-20240413 
Sat Apr 13 17:52:17 2024 : sellside/dongfang.l2_chars from 20240404 to 20240413, total 1 periods
Start sellside/dongfang.l2_chars:20240404-20240413 
Done sellside/dongfang.l2_chars:20240404-20240413 
Sat Apr 13 17:55:05 2024 : sellside/dongfang.ms_chars from 20240404 to 20240413, total 1 periods
Start s

In [1]:
from run_model import ModelPred

model_preds = [
    ModelPred('gru_day'    , 'swalast' , 0 , 'gru_day_V0') ,
    ModelPred('gruRTN_day' , 'swalast' , 0 , 'gruRTN_day_V0') ,
    ModelPred('gruRES_day' , 'swalast' , 0 , 'gruRES_day_V0')
]
[md.get_df().deploy() for md in model_preds]

Load  2 DataBlocks ...... cost 0.06 secs
Align 2 DataBlocks ...... cost 0.09 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks ...... cost 0.09 secs
Align 2 DataBlocks ...... cost 0.08 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks ...... cost 0.08 secs
Align 2 DataBlocks ...... cost 0.09 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new


[None, None, None]

In [1]:
from run_model import ModelTest
mt = ModelTest.new('patch_tst')
mt.try_forward()

Load  2 DataBlocks ...... cost 0.06 secs
Align 2 DataBlocks ...... cost 0.13 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
[20231016 20240412]
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])


ModelOutputs(outputs=tensor([[ 0.0120],
        [ 0.0092],
        [ 0.0073],
        ...,
        [ 0.0230],
        [-0.0483],
        [-0.0653]], grad_fn=<AddmmBackward0>))